In [21]:
import dash

from dash import dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from folium.plugins import MarkerCluster
import folium
import requests
import json

from dotenv import load_dotenv
import os

In [14]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [15]:
from dash import State



# Load the data from the API
response = requests.get("https://us-central1-skillful-coast-423214-a6.cloudfunctions.net/lastEntries")
df_latest = pd.read_json(json.dumps(response.json()))

# Rename and clean up DataFrame columns
df_latest.columns = ["index", "time", "temperature", "luftfeuchtigkeit", "pm10", "pm2.5", "senseidfk", "r"]
df_latest.drop(columns="r", inplace=True)

# Connect to the PostgreSQL database to get station info
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# SQL SELECT query to get stations
select_stations_query = "SELECT ort, lat, lon, senseid FROM box_id;"
cursor.execute(select_stations_query)
stations = cursor.fetchall()

# Initialize the map centered on Germany
map_deutschland = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Create a Marker Cluster
marker_cluster = MarkerCluster().add_to(map_deutschland)

# Add weather stations to the map
for station in stations:
    ort, lat, lon, senseid = station
    # Filter the DataFrame for the current station
    station_data = df_latest[df_latest['senseidfk'] == senseid]
    
    # Create a popup with the station's current data
    if not station_data.empty:
        latest_data = station_data.iloc[-1]
        popup_text = (
            f"Station: {ort}<br>"
            f"Time: {latest_data['time']}<br>"
            f"Temperature: {latest_data['temperature']} °C<br>"
            f"Luftfeuchtigkeit: {latest_data['luftfeuchtigkeit']} %<br>"
            f"PM10: {latest_data['pm10']} µg/m³<br>"
            f"PM2.5: {latest_data['pm2.5']} µg/m³"
        )
    else:
        popup_text = f"Station: {ort}<br>No data available."
    
    folium.Marker(
        location=[lat, lon],
        popup=popup_text
    ).add_to(marker_cluster)

# Display the map
map_deutschland.save('map_deutschland.html')



connection.close()
station_data

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_11904\343636518.py:7: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Verbindung zur Datenbank erfolgreich hergestellt.


,index,time,temperature,luftfeuchtigkeit,pm10,pm2.5,senseidfk
6,1735607,"Thu, 27 Jun 2024 13:10:05 GMT",29.02,58.56,11.95,5.63,64722d1c9be0580007f776d9


In [16]:
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage für die Stationen
select_value_query = """
    SELECT v.*, b.ort
    FROM value v
    JOIN box_id b ON v.senseid_fk = b.senseid;
"""
cursor.execute(select_value_query)
values = cursor.fetchall()

df = pd.DataFrame(values, columns=[desc[0] for desc in cursor.description])

cursor.close()
connection.close()

Verbindung zur Datenbank erfolgreich hergestellt.


Werte Droppen

In [17]:
df = df[df['luftfeuchtigkeit'] != 100.00]
df = df[df['luftfeuchtigkeit'] != 99.90]
df = df[df["pm10"]<1200]
bedingung = df['temperature'] > df['temperature'].quantile(0.99)
df = df.drop(df[bedingung].index)

In [18]:
stations = df['ort'].unique()
stations_options = [{'label': 'Alle Stationen', 'value': 'all'}] + [{'label': station, 'value': station} for station in stations]

In [19]:
stations

array(['Ettlingen', 'Stuttgart', 'München', 'Berlin', 'Hamburg',
       'Dortmund', 'Jena', 'Mersch'], dtype=object)

In [26]:
# Fetch the data from the API

app = dash.Dash(__name__)
app.layout = html.Div(id='app-container', children=[
    html.H1(children='Umweltüberwachung Dashboard', id='title'),
    
    html.Button('Kartenansicht', id='map-view-button', n_clicks=0, style={'margin-right': '10px'}),
    html.Button('Kennzahlenansicht', id='metrics-view-button', n_clicks=0),

        html.Button('Toggle Dark/Light Mode', id='toggle-button', n_clicks=0, style={'margin-top': '20px'}),
    
    html.Div(id='control-container', children=[
        html.Div(
            children='Wählen Sie eine Station und eine Kennzahl aus, um die Daten anzuzeigen.',
            id='description',
        ),

        dcc.Dropdown(
            id='station-dropdown',
            options=stations_options,
            value='all',
        ),

        html.Div(
            children='Wählen Sie eine Kennzahl:',
            id='metric-label',
            style={'marginTop': 20}
        ),
        
        dcc.RadioItems(
            id='metric-radio',
            options=[
                {'label': 'Durchschnitt', 'value': 'mean'},
                {'label': 'Median', 'value': 'median'},
                {'label': 'Varianz', 'value': 'var'},
                {'label': 'Standardabweichung', 'value': 'std'},
                {'label': 'Min', 'value': 'min'},
                {'label': 'Max', 'value': 'max'},
                {'label': 'Boxplot', 'value': 'boxplot'}
            ],
            value='mean',
            labelStyle={'display': 'inline-block', 'marginRight': 10}
        ),

        html.Div(
            children='Wählen Sie die Aggregationsstufe:',
            id='aggregation-label',
            style={'marginTop': 20}
        ),
        
        dcc.Checklist(
            id='aggregation-checklist',
            options=[
                {'label': 'Jahr', 'value': 'year'},
                {'label': 'Monat', 'value': 'month'},
                {'label': 'Tag', 'value': 'day'},
                {'label': 'Stunde', 'value': 'hour'}
            ],
            value=['year', 'month'],
            labelStyle={'display': 'inline-block', 'marginRight': 10}
        ),
        
        html.Div(
            children='Anzeigen der letzten 7 Tage:',
            id='last-7-days-label',
            style={'marginTop': 20}
        ),
        
        dcc.Checklist(
            id='last-7-days-checklist',
            options=[
                {'label': 'Letzte 7 Tage', 'value': 'last_7_days'}
            ],
            value=[],
            labelStyle={'display': 'inline-block', 'marginRight': 10}
        )
    ]),


# Layout der Dash-App definieren

    html.Div(id='view-container', children=[
        dcc.Graph(id='temperature-graph'),
        dcc.Graph(id='luftfeuchtigkeit-graph'),
        dcc.Graph(id='pm-graph'),
    ])

    
])

# Funktion zur Berechnung von IQR
def calculate_metrics(group, metric):
    if metric == 'iqr':
        return group.quantile(0.75) - group.quantile(0.25)
    else:
        return group.agg(metric)
    
@app.callback(
    [Output('view-container', 'children'),
     Output('control-container', 'style')],
    [Input('map-view-button', 'n_clicks'), Input('metrics-view-button', 'n_clicks')]
)
def switch_views(map_view_clicks, metrics_view_clicks):
    if map_view_clicks > metrics_view_clicks:
        return [html.Iframe(srcDoc=open('map_deutschland.html', 'r').read(), width='100%', height='600')], {'display': 'none'}
    else:
        return [
            dcc.Graph(id='temperature-graph'),
            dcc.Graph(id='luftfeuchtigkeit-graph'),
            dcc.Graph(id='pm-graph')
        ], {'display': 'block'}


# Callback zur Aktualisierung der Diagramme
@app.callback(
    [Output('temperature-graph', 'figure'),
     Output('luftfeuchtigkeit-graph', 'figure'),
     Output('pm-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('metric-radio', 'value'),
     Input('aggregation-checklist', 'value'),
     Input('last-7-days-checklist', 'value'),
     Input('toggle-button', 'n_clicks')]
)

def update_graphs(selected_station, selected_metric, selected_aggregation, last_7_days, n_clicks):
    if not selected_station or not selected_metric:
        raise PreventUpdate

    # Filter the DataFrame by the selected station
    if selected_station != 'all':
        filtered_df = df[df['ort'] == selected_station]
    else:
        filtered_df = df.copy()

    # Clean the DataFrame of NaN values
    filtered_df = filtered_df.dropna(subset=['temperature', 'luftfeuchtigkeit', 'pm2_5', 'pm10'])

    # Filter the data for the last 7 days if selected
    if 'last_7_days' in last_7_days:
        last_7_days_date = filtered_df['time'].max() - pd.Timedelta(days=7)
        filtered_df = filtered_df[filtered_df['time'] >= last_7_days_date]

        temp_fig = px.line(filtered_df, x='time', y='temperature', title='Temperatur der letzten 7 Tage')
        humidity_fig = px.line(filtered_df, x='time', y='luftfeuchtigkeit', title='Luftfeuchtigkeit der letzten 7 Tage')
        pm_fig = px.line(filtered_df, x='time', y=['pm2_5', 'pm10'], title='PM2.5 und PM10 der letzten 7 Tage')

    elif selected_metric == 'boxplot':
        # Create Boxplots for the entire DataFrame
        temp_fig = px.box(filtered_df, y='temperature', title='Temperatur Boxplot')
        humidity_fig = px.box(filtered_df, y='luftfeuchtigkeit', title='Luftfeuchtigkeit Boxplot')
        pm_fig = px.box(filtered_df, y=['pm2_5', 'pm10'], title='PM2.5 und PM10 Boxplot')

    else:
        # Ensure valid aggregation combinations only if not 'boxplot'
        if 'hour' in selected_aggregation and 'day' not in selected_aggregation:
            selected_aggregation.append('day')
        if 'day' in selected_aggregation and 'month' not in selected_aggregation:
            selected_aggregation.append('month')
        if 'month' in selected_aggregation and 'year' not in selected_aggregation:
            selected_aggregation.append('year')

        # Create the aggregation column based on the selection
        aggregation = ''
        if 'year' in selected_aggregation:
            aggregation += '%Y'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

        if 'month' in selected_aggregation:
            aggregation += '-%m'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
            filtered_df['aggregation'] = pd.to_datetime(filtered_df['aggregation'], format='%Y-%m')
            filtered_df['aggregation'] = filtered_df['aggregation'].dt.strftime('%b. %Y')

        if 'day' in selected_aggregation:
            aggregation += '-%d'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

        if 'hour' in selected_aggregation:
            aggregation += ' %H'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
        if aggregation == '-%d':
            filtered_df['aggregation'] = filtered_df['time'].dt.day_of_year

        if 'month' in selected_aggregation:
            # Create a list of all possible months within the data range
            all_months = pd.date_range(start=filtered_df['time'].min().replace(day=1), end=filtered_df['time'].max(), freq='MS').strftime('%b. %Y').tolist()
            # Drop duplicates to avoid reindex issues
            filtered_df = filtered_df.drop_duplicates(subset=['aggregation'])
            # Reindex the DataFrame to include all months
            filtered_df = filtered_df.set_index('aggregation').reindex(all_months).reset_index()

        # Plot based on the selected metric
        aggregation_mapping = {
            'mean': 'mean',
            'median': 'median',
            'var': 'var',
            'std': 'std',
            'min': 'min',
            'max': 'max'
        }

        temp_fig = px.line(filtered_df.groupby('aggregation').agg({'temperature': aggregation_mapping[selected_metric]}).reset_index(), 
                            x='aggregation', y='temperature', title=f'Temperatur ({selected_metric.capitalize()})')
        temp_fig.update_traces(line=dict(color='red'))
        humidity_fig = px.line(filtered_df.groupby('aggregation').agg({'luftfeuchtigkeit': aggregation_mapping[selected_metric]}).reset_index(), 
                                x='aggregation', y='luftfeuchtigkeit', title=f'Luftfeuchtigkeit ({selected_metric.capitalize()})')
        humidity_fig.update_traces(line=dict(color='blue'))
        pm_fig = px.line(filtered_df.groupby('aggregation').agg({'pm2_5': aggregation_mapping[selected_metric], 'pm10': aggregation_mapping[selected_metric]}).reset_index(), 
                        x='aggregation', y=['pm2_5', 'pm10'], title=f'PM2.5 und PM10 ({selected_metric.capitalize()})')
        pm_fig.for_each_trace(lambda trace: trace.update(line=dict(color='blue')) if trace.name == 'pm2_5' else trace.update(line=dict(color='red')))

    if n_clicks % 2 == 0:
        graph_layout = dict(plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'))
        temp_fig.update_layout(graph_layout)
        humidity_fig.update_layout(graph_layout)
        pm_fig.update_layout(graph_layout)
    else:
        graph_layout = dict(plot_bgcolor='white', paper_bgcolor='white', font=dict(color='black'))
        temp_fig.update_layout(graph_layout)
        humidity_fig.update_layout(graph_layout)
        pm_fig.update_layout(graph_layout)

    return temp_fig, humidity_fig, pm_fig

# Callback zur Umschaltung des Modus
@app.callback(
    [Output('title', 'style'),
     Output('toggle-button', 'style'),
     Output('description', 'style'),
     Output('metric-label', 'style'),
     Output('aggregation-label', 'style'),
     Output('last-7-days-label', 'style'),
     Output('station-dropdown', 'style'),
     Output('metric-radio', 'labelStyle'),
     Output('aggregation-checklist', 'labelStyle'),
     Output('last-7-days-checklist', 'labelStyle'),
     Output('app-container', 'style')],
    [Input('toggle-button', 'n_clicks')]
)
def toggle_dark_light_mode(n_clicks):
    if n_clicks % 2 == 0:
        dark_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        light_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        dropdown_style = {'backgroundColor': 'black', 'color': 'white'}
        container_style = {'backgroundColor': 'black'}
    else:
        dark_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        light_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        dropdown_style = {'backgroundColor': 'white', 'color': 'black'}
        container_style = {'backgroundColor': 'white'}

    return (dark_mode_style, light_mode_style, dark_mode_style, dark_mode_style,
            dark_mode_style, dark_mode_style, dropdown_style, light_mode_style, 
            light_mode_style, light_mode_style, container_style)

@app.callback(
    [Output('aggregation-checklist', 'options'),
     Output('aggregation-checklist', 'value')],
    [Input('metric-radio', 'value')]
)
def toggle_aggregation_checklist(selected_metric):
    if selected_metric == 'boxplot':
        return [{'label': 'Jahr', 'value': 'year', 'disabled': True},
                {'label': 'Monat', 'value': 'month', 'disabled': True},
                {'label': 'Tag', 'value': 'day', 'disabled': True},
                {'label': 'Stunde', 'value': 'hour', 'disabled': True}], []
    else:
        return [{'label': 'Jahr', 'value': 'year', 'disabled': False},
                {'label': 'Monat', 'value': 'month', 'disabled': False},
                {'label': 'Tag', 'value': 'day', 'disabled': False},
                {'label': 'Stunde', 'value': 'hour', 'disabled': False}], ['year', 'month']
# App ausführen
if __name__ == '__main__':
    app.run_server(debug=True,host='127.0.0.1', port=8050)
